# Form the NB Regression Random Intercept Model: Simulated set

Using Block update of beta and MM-update. We will use the file "fit_old.jl"

Without having to compute the gradient and hessian of variance components vector, we finish in 91 iterations and 11 seconds.

Next we may try joint estimation using Newton's + IPOPT. 


    note: Here I give r = 10 when creating the model object, this way we are not having to update r at each iteration. 

In [1]:
using Revise
using DataFrames, Random, GLM, GLMCopula
using ForwardDiff, Test, LinearAlgebra
using LinearAlgebra: BlasReal, copytri!
using SpecialFunctions

Random.seed!(1234)

# sample size
N = 10000
# observations per subject
n = 5

variance_component_1 = 0.1
variance_component_2 = 0.1

r = 10
p = 0.7
μ = r * (1-p) * inv(p)

# var = r * (1-p) * inv(p^2)

# true beta
β_true = log(μ)

dist = NegativeBinomial

Γ = variance_component_1 * ones(n, n) + variance_component_2 * Matrix(I, n, n)
vecd = [dist(r, p) for i in 1:n]
nonmixed_multivariate_dist = NonMixedMultivariateDistribution(vecd, Γ)

Y_Nsample = simulate_nobs_independent_vectors(nonmixed_multivariate_dist, N)

10000-element Array{Array{Float64,1},1}:
 [2.0, 1.0, 2.0, 6.0, 1.0]
 [8.0, 5.0, 3.0, 5.0, 2.0]
 [2.0, 4.0, 4.0, 9.0, 2.0]
 [4.0, 5.0, 2.0, 4.0, 1.0]
 [9.0, 10.0, 10.0, 1.0, 1.0]
 [4.0, 4.0, 3.0, 4.0, 3.0]
 [9.0, 6.0, 5.0, 4.0, 4.0]
 [6.0, 7.0, 3.0, 6.0, 0.0]
 [4.0, 7.0, 3.0, 3.0, 3.0]
 [4.0, 4.0, 3.0, 3.0, 6.0]
 [2.0, 6.0, 5.0, 2.0, 3.0]
 [3.0, 2.0, 2.0, 7.0, 4.0]
 [2.0, 3.0, 6.0, 2.0, 3.0]
 ⋮
 [2.0, 3.0, 1.0, 4.0, 2.0]
 [1.0, 3.0, 6.0, 6.0, 2.0]
 [2.0, 1.0, 3.0, 5.0, 7.0]
 [6.0, 3.0, 1.0, 1.0, 5.0]
 [2.0, 3.0, 4.0, 4.0, 9.0]
 [2.0, 3.0, 6.0, 2.0, 6.0]
 [2.0, 3.0, 1.0, 1.0, 2.0]
 [8.0, 0.0, 6.0, 3.0, 5.0]
 [5.0, 3.0, 2.0, 3.0, 6.0]
 [5.0, 3.0, 3.0, 8.0, 5.0]
 [8.0, 2.0, 1.0, 4.0, 3.0]
 [3.0, 6.0, 1.0, 3.0, 3.0]

In [2]:
d = NegativeBinomial(10, 0.5)
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

In [3]:
d = NegativeBinomial(10) # set r here
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
@show gcm.β
@show gcm.Σ
@show gcm.data[1].d.r

initializing β using Newton's Algorithm under Independence Assumption
1 0.0 -117067.2665631555 39999
2 -117067.2665631555 -117067.2665631555 39999
initializing variance components using MM-Algorithm
gcm.β = [1.4871979594607563]
gcm.Σ = [0.09147887350489728, 0.05454936551160692]


2-element Array{Float64,1}:
 0.09147887350489728
 0.05454936551160692

In [4]:
β_true

1.4552872326068422

Initialize β and σ2, here I just copy the solution for β and σ2 from MixedModels.jl over

In [3]:
GLMCopula.loglikelihood!(gcm, true, true)

-116743.32998423115

## Fit_old (not updating r) works

If true $r$ is not 1, setting the correct $r$ in initialize function will make it work too.

In [12]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 500, mehrotra_algorithm="yes", warm_start_init_point="yes", hessian_approximation = "exact"))

gcm.β = [1.4871979594607563]
gcm.Σ = [0.09147889287065758, 0.05454830925214411]
gcm.β = [1.4871979594607563]
gcm.Σ = [0.09147891165676356, 0.05454728469513783]

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        1

Total number of variables............................:        1
                     variables with only lower bounds

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  40  1.1670856e+05 0.00e+00 4.20e-05 -11.0 1.09e-08    -  1.00e+00 1.00e+00f  1
gcm.β = [1.4600653679395466]
gcm.Σ = [0.09935916129677433, 0.08800391081026562]
  41  1.1670856e+05 0.00e+00 3.80e-05 -11.0 9.85e-09    -  1.00e+00 1.00e+00f  1
gcm.β = [1.4600653590402208]
gcm.Σ = [0.09935916494487544, 0.08800399621345237]
  42  1.1670856e+05 0.00e+00 3.43e-05 -11.0 8.90e-09    -  1.00e+00 1.00e+00f  1
gcm.β = [1.4600653510023993]
gcm.Σ = [0.09935916823981635, 0.08800407334916008]
  43  1.1670856e+05 0.00e+00 3.10e-05 -11.0 8.04e-09    -  1.00e+00 1.00e+00f  1
gcm.β = [1.4600653437426843]
gcm.Σ = [0.09935917121578826, 0.08800414301771696]
  44  1.1670856e+05 0.00e+00 2.80e-05 -11.0 7.26e-09    -  1.00e+00 1.00e+00h  1
gcm.β = [1.4600653371857504]
gcm.Σ = [0.09935917390366869, 0.08800420594197564]
  45  1.1670856e+05 0.00e+00 2.53e-05 -11.0 6.56e-09    -  1.00e+00 1.00e+00f  1
gcm.β = [1.4600653312635656]
gcm.

  89  1.1670856e+05 0.00e+00 2.86e-07 -11.0 7.43e-11    -  1.00e+00 1.00e+00f  1
gcm.β = [1.4600652766361608]
gcm.Σ = [0.09935919872494357, 0.08800478701244416]
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90  1.1670856e+05 0.00e+00 2.59e-07 -11.0 6.71e-11    -  1.00e+00 1.00e+00f  1
gcm.β = [1.4600652765755395]
gcm.Σ = [0.09935919874979308, 0.08800478759420219]
  91  1.1670856e+05 0.00e+00 2.34e-07 -11.0 6.06e-11    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 91

                                   (scaled)                 (unscaled)
Objective...............:   4.5124357909384134e+03    1.1670856227598549e+05
Dual infeasibility......:   2.3358297896438130e-07    6.0413344167287164e-06
Constraint violation....:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity.........:   0.0000000000000000e+00    0.0000000000000000e+00
Overall NLP error.......:   2.3358297896438130e-07    6.0413344167287164e-06


Number of objective functio

In [13]:
@show β_true
@show gcm.β
@show gcm.Σ
@show variance_component_1, variance_component_2
@show gcm.∇β
@show gcm.data[1].d
@show GLMCopula.loglikelihood!(gcm, true, true);

β_true = 1.4552872326068422
gcm.β = [1.4600652765755395]
gcm.Σ = [0.09935919874979308, 0.08800478759420219]
gcm.∇β = [-6.041334416728716e-6]
GLMCopula.loglikelihood!(gcm, true, true) = -116708.56227598326


## fit_old (updating r)

In [21]:
d = NegativeBinomial() # r is 1
link = LogLink()
D = typeof(d)
Link = typeof(link)
T = Float64
gcs = Vector{GLMCopulaVCObs{T, D, Link}}(undef, N)
for i in 1:N
    y = Float64.(Y_Nsample[i])
    X = ones(n, 1)
    V = [ones(n, n), Matrix(I, n, n)] # V[1] isn't positive definite!!!!
#     A = randn(n, n)
#     V = [A'*A, Matrix(I, n, n)]
    gcs[i] = GLMCopulaVCObs(y, X, V, d, link)
end
gcm = GLMCopulaVCModel(gcs);

initialize_model!(gcm)
@show gcm.β
@show gcm.Σ
@show gcm.data[1].d.r


estimated β = 1.4600652765755395; true β = 1.4552872326068422
estimated variance component 1 = 0.09935919874979308; true variance component 1 = 0.1
estimated variance component 2 = 0.08800478759420219; true variance component 2 = 0.1


8.00000194402719

In [9]:
@time fit2!(gcm, IpoptSolver(print_level = 5, max_iter = 50, 
    mehrotra_algorithm="yes", warm_start_init_point="yes", hessian_approximation = "exact"))


BenchmarkTools.Trial: 
  memory estimate:  18.31 MiB
  allocs estimate:  200000
  --------------
  minimum time:     37.735 ms (0.00% GC)
  median time:      41.235 ms (0.00% GC)
  mean time:        41.593 ms (2.35% GC)
  maximum time:     48.708 ms (0.00% GC)
  --------------
  samples:          121
  evals/sample:     1